## OBTAINING ALL TWEETS FROM A USER

In [1]:
import requests
import json
import pandas as pd
import tweepy

In [2]:
#pip install --upgrade tweepy  # needed v.4

In [3]:
# LA LLAVE
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAF02cQEAAAAAAw4Ko%2BlvaP%2FEKo4jetBjHN%2BGEpY%3D8ot7Zeg7DytI7xLRXuZO23cSWSLBIAHPMKrw9b8jebi9j7nWsI"

#Extracción de data a partir del @username.
Basado en Tweepy v4.9.0

In [4]:
input_user = 'mferna'

In [5]:
def extract_tweet_data(input_user):
    """
    Extracts data from a Twitter account.

    Parameters:
    input_user: str
        Twitter account to be analysed (without the @).

    Returns:
    pd.DataFrame
        A dataframe of the user tweets containing, time of creation,
        text, tweet id, number of retweets and number of likes.
    """
    # Authorize access
    client = tweepy.Client(BEARER_TOKEN)

    # Get user ID from username
    user_id = client.get_user(username=input_user).data.id

    # Get Tweets timeline (check all pages, max 100 tweets per page, max 32 pages)
    df_list = []
    df_len0 = []
    df_len = []
    for response in tweepy.Paginator(client.get_users_tweets, user_id,
                                    tweet_fields='created_at,lang,public_metrics',
                                    max_results=100, limit=32):
        page_df = pd.DataFrame.from_dict(response.data)
        df_len0.append(len(page_df))
        # Delete non-spanish tweets
        page_df = page_df.drop(page_df[page_df.lang !='es'].index)
        page_df = page_df.drop(['lang'], axis=1)
        df_len.append(len(page_df))
        # Organize columns
        retweet_count = []
        like_count = []
        for item in page_df['public_metrics'].tolist():
          retweet_count.append(item['retweet_count'])
          like_count.append(item['like_count'])
        page_df = page_df.drop(['public_metrics'], axis=1)
        page_df['retweet_count'] = retweet_count
        page_df['like_count'] = like_count
        df_list.append(page_df)
    es_tweets = round(sum(df_len)*100/sum(df_len0),1)
    print(f'Obtained {es_tweets}% tweets in spanish from total of {sum(df_len0)} tweets from user @{input_user}.')
    tweets_df = pd.concat(df_list, ignore_index=True)
    return tweets_df

OLD EXTRACTION CODE (only works for obtaining user info, and only 100 tweets max)\
Es una modificación de los ejemplos para obtener [info del user](https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Lookup/get_users_with_bearer_token.py) y [sus tweets](https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Tweet-Timeline/user_tweets.py)


In [6]:
def create_url(username, twitter_object):
    if twitter_object == 'user':
      usernames = f'usernames={input_user}'
      user_fields = 'user.fields=description,created_at,protected,public_metrics,verified'
      url = f'https://api.twitter.com/2/users/by?{usernames}&{user_fields}'
    elif twitter_object == 'tweet':
      user_dict = get_json('user')
      user_id = user_dict['data'][0]['id']
      url = f'https://api.twitter.com/2/users/{user_id}/tweets?max_results=100'
    else:
      print('Error. Wrong Twitter Object.')
      return None
    return url


def get_params(twitter_object):
    if twitter_object == 'tweet':
      params = {"tweet.fields": "created_at,lang,public_metrics"}
    else:
      print('Error. Wrong Twitter Object.')
      return None
    return params


def bearer_oauth_user(r):
    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r


def bearer_oauth_tweet(r):
    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r


def connect_to_endpoint(url, twitter_object):
    if twitter_object == 'user':
      response = requests.request("GET", url, auth=bearer_oauth_user,)
    elif twitter_object == 'tweet':
      params = get_params('tweet')
      response = requests.request("GET", url, auth=bearer_oauth_tweet,params=params)
    else:
        print('Error. Wrong Twitter Object.')
        return None
    if response.status_code != 200:
        raise Exception(f'Request returned an error: {response.status_code} {response.text}')
    return response.json()


def get_json(twitter_object):
    if twitter_object == 'user':
      url = create_url(input_user, 'user')
      json_response = connect_to_endpoint(url, 'user')
    elif twitter_object == 'tweet':
      url = create_url(input_user, 'tweet')
      json_response = connect_to_endpoint(url, 'tweet')
    else:
        print('Error. Wrong Twitter Object.')
        return None
    return json_response


def extract_user_data(username):  # MAIN FILE
    """
    Extracts data from a Twitter account.

    Parameters:
    input_user: str
        Twitter account to be analysed (without the @).
    Returns:
    dict
        A dict containing the releveant variables about the user
    """
    # Get user ID from username
    user_dict = get_json('user')
    #user_id = user_dict['data'][0]['id']

    # Get Tweets timeline
    #tweet_dict = get_json('tweet')
    #tweet_pd = pd.DataFrame.from_dict(tweet_dict['data'])
    return user_dict

#OUTPUTS:

In [7]:
extract_user_data(input_user)  ## Dict with user info

{'data': [{'verified': False,
   'created_at': '2008-04-02T01:08:53.000Z',
   'description': 'Médico-Epidemíologo, Doctor en Educación, Profesor Titular,Coordinador de Investigaciòn Facultad Med UCV. hablamos de TIC en Investigación, Educación y Salud.',
   'public_metrics': {'followers_count': 6093,
    'following_count': 2968,
    'tweet_count': 58147,
    'listed_count': 300},
   'username': 'mferna',
   'id': '14279589',
   'name': 'Mariano Fernández S',
   'protected': False}]}

In [8]:
extract_tweet_data(input_user)  ## DataFrame with all tweets in spanish from user

Obtained 73.3% tweets in spanish from total of 3198 tweets from user @mferna.


,created_at,id,text,retweet_count,like_count
0,2022-05-03 01:37:08+00:00,1521302809450369026,Guía de evaluación para la conservación de dat...,2,1
1,2022-04-27 15:56:31+00:00,1519344752981909504,RT @gorka_orive: Os recomiendo seguir la infor...,115,0
2,2022-04-27 15:56:09+00:00,1519344657590800391,RT @DrPontecarlosi: Nuevas guías sobre uso de ...,74,0
3,2022-04-27 14:56:02+00:00,1519329529474555905,Sobre la Hepatitis Infantil Desconocida... htt...,0,0
4,2022-04-26 01:20:00+00:00,1518761781325451266,"Adenovirus 41 o covid, sospechosos tras las he...",1,0
...,...,...,...,...,...
2339,2019-10-02 14:51:45+00:00,1179408633756618754,RT @campussanofi: 👍👏 @AbbottGlobal y @sanofi s...,9,0
2340,2019-09-30 23:32:52+00:00,1178815001123065861,RT @opsoms: CONVOCATORIA DE ARTÍCULOS ❗️\n\n📘 ...,13,0
2341,2019-09-30 00:23:19+00:00,1178465311672143874,RT @biotecfarffucv: En #Biotecfar #SalvamosVid...,81,0
2342,2019-09-30 00:19:54+00:00,1178464452481605638,Cuántas tazas de café hay que tomar al día par...,0,0


#Procesamiento de data

In [10]:
def vars_word(word, input_df):
    """
    Compute the variables for a given word.

    Parameters:
    word: str
        Word to be analysed in the timeline tweets.
    input_df: pd.DataFrame
        A dataframe of the user tweets containing, time of creation,
        text, tweet id, number of retweets and number of likes.

    Returns:
    tuple(float, float, float, float, float):
        A tuple containing variables of the word: frequency, like rate, retweet
        rate, popularity, and polemicity
    tuple(float, float, float):
        A tuple containing counters of the word: number of times it appears
        on all the tweets, number of tweets it appears in, number of retweets it
        appears in.
    """

    df = input_df.copy()
    # Flatten text column
    splitted_tweets=df.clean_text.apply(lambda a: a.split()).tolist()#.count("datos")
    # Flatten text column
    total_word_list = [word_tweet for word_list in splitted_tweets for word_tweet in word_list]
    

    word_count = total_word_list.count(word)
    total_word_count = len(total_word_list)
    freq = word_count/total_word_count

    df['word_count'] = df.clean_text.map(lambda t: t.count(word))
    tweets_containing = df['word_count'].astype(bool).sum()
    #if tweets_containing == 0:
    #    return None
    df.drop(['word_count'], axis=1)
    # like_count_flag
    like_count = df[df['word_count'].astype(bool)]['like_count'].sum()
    retweet_count = df[df['word_count'].astype(bool)]['retweet_count'].sum()
    #if (like_count == 0) or (retweet_count == 0):
    #    return None
    like_rate = like_count/tweets_containing
    retweet_rate = retweet_count/tweets_containing

    popularity  = retweet_rate/freq
    polemicity  = retweet_rate/like_rate

    variables = (freq, like_rate, retweet_rate, popularity, polemicity)
    counters = (word_count, like_count, retweet_count)
    return variables, counters


def vars_cat(category, df):
    """
    Compute the variables for a given category.

    Parameters:
    category: list
        List of words to be analysed in group in the timeline tweets.
    df: pd.DataFrame
        A dataframe of the user tweets containing, time of creation,
        text, tweet id, number of retweets and number of likes.

    Returns:
    tuple(float, float, float):
        A tuple containing counters of the words in the category: number of
        times they appear on all the tweets, number of tweets they appears in,
        number of retweets they appear in.
    tuple(float, float, float):
        A tuple containing popularity and polemicity of the category.
    """

    category = [word for word in category if word]
    pop_cat = pol_cat = word_count_cat = like_count_cat = retweet_count_cat = 0
    for word in category:
        #if vars_word(word, df) is None:
        vars, counters = vars_word(word, df)
        pop_cat += counters[0] * vars[3]
        pol_cat += counters[0] * vars[4]
        word_count_cat += counters[0]
        like_count_cat += counters[1]
        retweet_count_cat += counters[2]

    pop_cat = pop_cat / word_count_cat
    pol_cat = pol_cat / word_count_cat

    cat_counters = (word_count_cat, like_count_cat, retweet_count_cat)
    pop_pol_cats = (pop_cat, pol_cat)
    return cat_counters, pop_pol_cats

#OUTPUTS

## EDA

In [14]:
import itertools


import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy

In [15]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text, stem="None"):
    try:

        final_string = ""

        # minusculas
        text = text.lower()
        text = text.replace(" u "," ").replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")

        # quitar salto de lineas
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
        text=emoji_pattern.sub(r'', text)
        # quitar signos gramaticales
        translator = str.maketrans('', '', string.punctuation)
        text = text.translate(translator)

        # stopwords
        text = text.split()
        useless_words = nltk.corpus.stopwords.words("spanish")
        useless_words = useless_words + ['...','considero','tal','vez','seria','debe','tener','siento', "dar", "hacer"
                            'estan','tan','parece','ademas','debido','cuenta','hace','cada','toda','si', "ser",
                                         "ma", "mas", "más","bien", "buena", "creo", "aun"]
        # 
        text_filtered = [word for word in text if not word in useless_words]

        # quitar numeros
        text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]
        

        # Lematizacion
        #if stem == 'Stem':
        #    stemmer = PorterStemmer() 
        #    text_stemmed = [stemmer.stem(y) for y in text_filtered]
        #elif stem == 'Lem':
        #    lem = WordNetLemmatizer()
        #    text_stemmed = [lem.lemmatize(y) for y in text_filtered]
        #elif stem == 'Spacy':
        #    text_filtered = nlp(' '.join(text_filtered))
        #    text_stemmed = [y.lemma_ for y in text_filtered]
        #else:
        #    text_stemmed = text_filtered

        #final_string = ' '.join(text_stemmed)
        final_string = ' '.join(text_filtered)

        # sinonimia
        #final_string = final_string.replace('docent','profesor')
    except AttributeError:
        final_string = ""

    return final_string

In [16]:
def ngrams_wrapper_1(sent):
    return list(nltk.ngrams(sent, 1))

def ngrams_wrapper_2(sent):
    return list(nltk.ngrams(sent, 2))

def ngrams_wrapper_3(sent):
    return list(nltk.ngrams(sent, 3))

In [17]:
def make_list_bigrams_1(text):
    sentences = nltk.sent_tokenize(text)
    tokenized = map(nltk.tokenize.word_tokenize, sentences)
    bigrams = map(ngrams_wrapper_1, tokenized)
    bigram = list(itertools.chain.from_iterable(bigrams))
    return bigram

def make_list_bigrams_2(text):
    sentences = nltk.sent_tokenize(text)
    tokenized = map(nltk.tokenize.word_tokenize, sentences)
    bigrams = map(ngrams_wrapper_2, tokenized)
    bigram = list(itertools.chain.from_iterable(bigrams))
    return bigram

def make_list_bigrams_3(text):
    sentences = nltk.sent_tokenize(text)
    tokenized = map(nltk.tokenize.word_tokenize, sentences)
    bigrams = map(ngrams_wrapper_3, tokenized)
    bigram = list(itertools.chain.from_iterable(bigrams))
    return bigram

In [18]:
def nlp_process_1(text):
    return '/'.join([str(i) for i in make_list_bigrams_1(clean_string(str(text), stem='Stem'))])

def nlp_process_2(text):
    return '/'.join([str(i) for i in make_list_bigrams_2(clean_string(str(text), stem='Stem'))])

def nlp_process_3(text):
    return '/'.join([str(i) for i in make_list_bigrams_3(clean_string(str(text), stem='Stem'))])
    

# Load Data

In [19]:
# dummy input (after cleaning)
o_df = extract_tweet_data(input_user)
#test_df = o_df.copy()
o_df

Obtained 73.3% tweets in spanish from total of 3198 tweets from user @mferna.


,created_at,id,text,retweet_count,like_count
0,2022-05-03 01:37:08+00:00,1521302809450369026,Guía de evaluación para la conservación de dat...,2,1
1,2022-04-27 15:56:31+00:00,1519344752981909504,RT @gorka_orive: Os recomiendo seguir la infor...,115,0
2,2022-04-27 15:56:09+00:00,1519344657590800391,RT @DrPontecarlosi: Nuevas guías sobre uso de ...,74,0
3,2022-04-27 14:56:02+00:00,1519329529474555905,Sobre la Hepatitis Infantil Desconocida... htt...,0,0
4,2022-04-26 01:20:00+00:00,1518761781325451266,"Adenovirus 41 o covid, sospechosos tras las he...",1,0
...,...,...,...,...,...
2339,2019-10-02 14:51:45+00:00,1179408633756618754,RT @campussanofi: 👍👏 @AbbottGlobal y @sanofi s...,9,0
2340,2019-09-30 23:32:52+00:00,1178815001123065861,RT @opsoms: CONVOCATORIA DE ARTÍCULOS ❗️\n\n📘 ...,13,0
2341,2019-09-30 00:23:19+00:00,1178465311672143874,RT @biotecfarffucv: En #Biotecfar #SalvamosVid...,81,0
2342,2019-09-30 00:19:54+00:00,1178464452481605638,Cuántas tazas de café hay que tomar al día par...,0,0


In [21]:
o_df["created_at"]=o_df["created_at"].apply(lambda a: pd.to_datetime(a).date())
o_df["clean_text"]=o_df["text"].apply(lambda x: clean_string(x))
o_df["bigram_key_words"]= o_df["clean_text"].apply(lambda x: nlp_process_2(x))
o_df["trigram_key_words"]= o_df["clean_text"].apply(lambda x: nlp_process_3(x))

o_df.head(10)

,created_at,id,text,retweet_count,like_count,clean_text,bigram_key_words,trigram_key_words
0,2022-05-03,1521302809450369026,Guía de evaluación para la conservación de dat...,2,1,guia evaluacion conservacion datos investigaci...,"('guia', 'evaluacion')/('evaluacion', 'conserv...","('guia', 'evaluacion', 'conservacion')/('evalu..."
1,2022-04-27,1519344752981909504,RT @gorka_orive: Os recomiendo seguir la infor...,115,0,rt gorkaorive recomiendo seguir informacion va...,"('rt', 'gorkaorive')/('gorkaorive', 'recomiend...","('rt', 'gorkaorive', 'recomiendo')/('gorkaoriv..."
2,2022-04-27,1519344657590800391,RT @DrPontecarlosi: Nuevas guías sobre uso de ...,74,0,rt drpontecarlosi nuevas guias uso aspirina pr...,"('rt', 'drpontecarlosi')/('drpontecarlosi', 'n...","('rt', 'drpontecarlosi', 'nuevas')/('drponteca..."
3,2022-04-27,1519329529474555905,Sobre la Hepatitis Infantil Desconocida... htt...,0,0,hepatitis infantil desconocida,"('hepatitis', 'infantil')/('infantil', 'descon...","('hepatitis', 'infantil', 'desconocida')"
4,2022-04-26,1518761781325451266,"Adenovirus 41 o covid, sospechosos tras las he...",1,0,adenovirus covid sospechosos tras hepatitis i...,"('adenovirus', 'covid')/('covid', 'sospechosos...","('adenovirus', 'covid', 'sospechosos')/('covid..."
5,2022-04-26,1518759259307225089,Hepatitis raras https://t.co/ooCqlbenPp,0,0,hepatitis raras httpstcooocqlbenpp,"('hepatitis', 'raras')/('raras', 'httpstcooocq...","('hepatitis', 'raras', 'httpstcooocqlbenpp')"
6,2022-04-24,1518231256831610881,Nuevas profesiones: la transformación de la re...,0,0,nuevas profesiones transformacion revolucion t...,"('nuevas', 'profesiones')/('profesiones', 'tra...","('nuevas', 'profesiones', 'transformacion')/('..."
7,2022-04-24,1518231226548731906,El futuro laboral del colectivo universitario ...,0,0,futuro laboral colectivo universitario – http...,"('futuro', 'laboral')/('laboral', 'colectivo')...","('futuro', 'laboral', 'colectivo')/('laboral',..."
8,2022-04-24,1518231170747711488,Presente y futuro de la realidad virtual | Enl...,0,0,presente futuro realidad virtual enlaces,"('presente', 'futuro')/('futuro', 'realidad')/...","('presente', 'futuro', 'realidad')/('futuro', ..."
9,2022-04-24,1518230665883435009,El juego de la escalera de las emociones ¡Cont...,0,0,juego escalera emociones ¡controla emociones h...,"('juego', 'escalera')/('escalera', 'emociones'...","('juego', 'escalera', 'emociones')/('escalera'..."


In [22]:
key_word_bigram = o_df[o_df['clean_text']!='']['bigram_key_words'].to_list()
all_key_word_bigram = '/'.join([str(i) for i in key_word_bigram])
df_all_key_word_bigram = pd.DataFrame(data={'bigrams': all_key_word_bigram.split('/')})
#DF_MOTIVO_DETRACTORES_ALL = DF_MOTIVO_DETRACTORES_ALL[DF_MOTIVO_DETRACTORES_ALL['bigrams']!="('buena', 'universidad')"]
#DF_MOTIVO_DETRACTORES_ALL = DF_MOTIVO_DETRACTORES_ALL[DF_MOTIVO_DETRACTORES_ALL['bigrams']!="('universidad', 'buena')"]
count_bigram_tweets = df_all_key_word_bigram.value_counts().reset_index()
count_bigram_tweets = count_bigram_tweets[count_bigram_tweets["bigrams"] != ""].reset_index(inplace=False).drop(['index'], axis=1)
count_bigram_tweets[0:20]

,bigrams,0
0,"('rt', 'doctormacias')",56
1,"('rt', 'gorkaorive')",49
2,"('educacion', 'distancia')",37
3,"('rt', 'saberucv')",30
4,"('miradorsalud', 've')",25
5,"('iberoamericana', 'educacion')",24
6,"('revista', 'iberoamericana')",23
7,"('ried', 'revista')",23
8,"('rt', 'sandralopezleon')",22
9,"('antilopino', '“')",20


In [23]:
key_word_trigram = o_df[o_df['clean_text']!='']['trigram_key_words'].to_list()
all_key_word_trigram = '/'.join([str(i) for i in key_word_trigram])
df_all_key_word_trigram = pd.DataFrame(data={'trigrams': all_key_word_trigram.split('/')})

count_trigram_tweets = df_all_key_word_trigram.value_counts().reset_index()
count_trigram_tweets = count_trigram_tweets[count_trigram_tweets["trigrams"] != ""].reset_index(inplace=False).drop(['index'], axis=1)
count_trigram_tweets[0:20]

,trigrams,0
0,"('iberoamericana', 'educacion', 'distancia')",24
1,"('ried', 'revista', 'iberoamericana')",23
2,"('revista', 'iberoamericana', 'educacion')",23
3,"('rt', 'antilopino', '“')",20
4,"('universidad', 'central', 'venezuela')",13
5,"('tendencias', 'pais', 'retina')",9
6,"('mit', 'technology', 'review')",8
7,"('technology', 'review', 'español')",8
8,"('america', 'latina', 'caribe')",7
9,"('repositorio', 'institucional', 'universidad')",7


#### Añadir visualización en el mismo notebook

In [24]:
o_df.head(3)

,created_at,id,text,retweet_count,like_count,clean_text,bigram_key_words,trigram_key_words
0,2022-05-03,1521302809450369026,Guía de evaluación para la conservación de dat...,2,1,guia evaluacion conservacion datos investigaci...,"('guia', 'evaluacion')/('evaluacion', 'conserv...","('guia', 'evaluacion', 'conservacion')/('evalu..."
1,2022-04-27,1519344752981909504,RT @gorka_orive: Os recomiendo seguir la infor...,115,0,rt gorkaorive recomiendo seguir informacion va...,"('rt', 'gorkaorive')/('gorkaorive', 'recomiend...","('rt', 'gorkaorive', 'recomiendo')/('gorkaoriv..."
2,2022-04-27,1519344657590800391,RT @DrPontecarlosi: Nuevas guías sobre uso de ...,74,0,rt drpontecarlosi nuevas guias uso aspirina pr...,"('rt', 'drpontecarlosi')/('drpontecarlosi', 'n...","('rt', 'drpontecarlosi', 'nuevas')/('drponteca..."


# EXCEL

In [26]:
with pd.ExcelWriter('Ngram_Analysis.xlsx') as writer:  
    o_df.to_excel(writer, sheet_name='Base')    
    count_trigram_tweets.to_excel(writer, sheet_name='Bigrams')
    count_bigram_tweets.to_excel(writer, sheet_name='Trigrams')

## NLP categorizacion

In [27]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
import pickle

## Oportunidades de Mejora

In [28]:
o_df_nouns=o_df[["clean_text"]].dropna().reset_index(inplace=False).dropna().drop('index', axis=1, inplace=False)

# We are going to create a document-term matrix using CountVectorizer, and exclude common Spanish stop words

lines_df = pd.read_csv('spanish.txt', sep="\t", header=None)
lines_df.columns = ["spanish_stopwords"]
lines=list(lines_df["spanish_stopwords"])
#lines


In [29]:
# Aplicaremos varios rounds de limpieza
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Segundo round
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text
'''
# Como no tenemos un Lemmatizer en español, hacemos manualmente algunas conversiones
# OJO: esto realmente no se hace a mano!!!

def detectadas(palabra):
    eliminar_s = ('libreros','textos','papelitos','monedas','páginas','anécdotas','perros','cuadernos','blogs',
                  'revistas','caballos','vecinos','madres','puntos','ricos','libros')
    if palabra in eliminar_s :
        return palabra[:-1]
    eliminar_es = ('mundiales','lectores','campeones','maníes','ustedes','autores')
    if palabra in eliminar_es:
        return palabra[:-2]
    return palabra

def clean_text_round3(text):
    return " ".join([detectadas(word) for word in text.split()])
    
'''
round1 = lambda x: clean_text_round1(x)
round2 = lambda x: clean_text_round2(x)
#round3 = lambda x: clean_text_round3(x)

In [30]:
# vemos la primer limpieza
data_clean = pd.DataFrame(o_df_nouns["clean_text"].apply(round1))
data_clean = pd.DataFrame(o_df_nouns["clean_text"].apply(round2))
#data_clean = pd.DataFrame(data_nouns["OportunidadesMejoraTextClean"].apply(round3))

# Esto es un nuevo campo por si quisieramos agregar alguna info adicional a cada año
# Nuestro caso repetimos los años, nos servirá para alguna visualización
full_names = list(data_clean.reset_index()["index"])#['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']

o_df_nouns['full_name'] = full_names# nombre de indices del comentario

# Lo guardamos como pickle
o_df_nouns.to_pickle("data_nouns.pkl")

cv = CountVectorizer(stop_words=lines)
data_cv = cv.fit_transform(data_clean["clean_text"])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index

# Lo guardamos como pickle
data_dtm.to_pickle("dtm.pkl")

# Lo guardamos como pickle también
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))


In [31]:
# Read in cleaned data
data_clean = pd.read_pickle('data_clean.pkl')

# Add new stop words
lines_df = pd.read_csv('spanish.txt', sep="\t", header=None)
lines_df.columns = ["spanish_stopwords"]
stop_words=list(lines_df["spanish_stopwords"])

# Recreate document-term matrix
cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean["clean_text"])
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = data_clean.index

# Pickle it for later use
pickle.dump(cv, open("cv_stop.pkl", "wb"))
data_stop.to_pickle("dtm_stop.pkl")
# Let's read in our document-term matrix
data = pd.read_pickle('dtm_stop.pkl')
data

,aba,abacantvreportes,abaratarlos,abarca,abbottglobal,abcedario,abdomen,abiert,abierta,abiertas,...,zelanda,zenecaoxford,zonas,zoom,zoonosis,zostavax,zoster,zotero,zulmacucunuba,zuluaga
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2340,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2341,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# One of the required inputs is a term-document matrix
tdm = data.transpose()
#tdm.tail()

# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())


## Análisis de temáticas

In [33]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.011*"rt" + 0.009*"educacion" + 0.006*"aprendizaje" + 0.005*"salud" + 0.004*"investigacion" + 0.004*"pais" + 0.003*"educativa" + 0.003*"futuro" + 0.003*"tendencias" + 0.003*"ciencia"'),
 (1,
  '0.032*"rt" + 0.007*"digital" + 0.004*"mundo" + 0.003*"vida" + 0.003*"coronavirus" + 0.003*"educacion" + 0.003*"enfermedad" + 0.003*"personas" + 0.003*"importante" + 0.003*"dia"')]

In [34]:
for idx, topic in lda.show_topics(formatted=False, num_words= 10):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))


Topic: 0 
Words: ['rt', 'educacion', 'aprendizaje', 'salud', 'investigacion', 'pais', 'educativa', 'futuro', 'tendencias', 'ciencia']
Topic: 1 
Words: ['rt', 'digital', 'mundo', 'vida', 'coronavirus', 'educacion', 'enfermedad', 'personas', 'importante', 'dia']


In [35]:
# LDA for num_topics = 3
lda_3 = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda_3.print_topics()

[(0,
  '0.015*"rt" + 0.014*"educacion" + 0.007*"digital" + 0.005*"tecnologia" + 0.005*"investigacion" + 0.004*"salud" + 0.004*"mejores" + 0.004*"revistas" + 0.004*"distancia" + 0.004*"datos"'),
 (1,
  '0.030*"rt" + 0.004*"dia" + 0.004*"google" + 0.004*"puede" + 0.004*"vida" + 0.004*"millones" + 0.003*"pais" + 0.003*"rigotordoc" + 0.003*"personas" + 0.003*"hacer"'),
 (2,
  '0.020*"rt" + 0.006*"aprendizaje" + 0.005*"coronavirus" + 0.005*"venezuela" + 0.005*"educacion" + 0.004*"ciencia" + 0.004*"acceso" + 0.004*"ideas" + 0.003*"saber" + 0.003*"futuro"')]

In [36]:
# LDA for num_topics = 4
lda_4 = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda_4.print_topics()

[(0,
  '0.031*"rt" + 0.007*"educacion" + 0.006*"venezuela" + 0.004*"enfermedad" + 0.004*"puede" + 0.004*"ciencia" + 0.004*"quiere" + 0.004*"digital" + 0.004*"salud" + 0.004*"mismo"'),
 (1,
  '0.025*"rt" + 0.007*"tecnologia" + 0.005*"digital" + 0.005*"educacion" + 0.004*"mundo" + 0.004*"asi" + 0.004*"aula" + 0.004*"salud" + 0.003*"menos" + 0.003*"ve"'),
 (2,
  '0.009*"rt" + 0.006*"aprendizaje" + 0.006*"educacion" + 0.005*"digital" + 0.005*"via" + 0.005*"herramientas" + 0.005*"universidad" + 0.005*"investigacion" + 0.004*"futuro" + 0.004*"educativa"'),
 (3,
  '0.021*"rt" + 0.008*"educacion" + 0.008*"pais" + 0.005*"articulos" + 0.005*"importante" + 0.004*"retina" + 0.004*"salud" + 0.004*"personas" + 0.004*"uso" + 0.004*"rigotordoc"')]

In [37]:
# LDA for num_topics = 10
lda_10 = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, passes=10)
lda_10.print_topics()

[(0,
  '0.037*"educacion" + 0.016*"distancia" + 0.010*"revista" + 0.009*"digitales" + 0.009*"ried" + 0.009*"rt" + 0.007*"pais" + 0.007*"digital" + 0.007*"puede" + 0.006*"mundo"'),
 (1,
  '0.011*"educativa" + 0.010*"rt" + 0.009*"digital" + 0.008*"innovacion" + 0.007*"big" + 0.007*"crear" + 0.007*"vida" + 0.007*"formacion" + 0.007*"dia" + 0.006*"castigo"'),
 (2,
  '0.024*"salud" + 0.022*"rt" + 0.013*"ciencia" + 0.010*"articulos" + 0.008*"abierta" + 0.007*"nunca" + 0.007*"mundial" + 0.007*"quiere" + 0.006*"datos" + 0.006*"excelente"'),
 (3,
  '0.027*"rt" + 0.007*"herramientas" + 0.006*"hacer" + 0.006*"estudios" + 0.006*"tic" + 0.006*"uso" + 0.006*"educativos" + 0.006*"respuesta" + 0.005*"asi" + 0.005*"academicos"'),
 (4,
  '0.012*"derecho" + 0.008*"necesario" + 0.008*"ejemplo" + 0.008*"buscan" + 0.008*"convierte" + 0.007*"todas" + 0.007*"mismo" + 0.007*"investigacion" + 0.006*"moviles" + 0.006*"experiencia"'),
 (5,
  '0.022*"rt" + 0.015*"mejores" + 0.013*"futuro" + 0.011*"ideas" + 0.010*"

### Ordenamiento por temáticas:

In [38]:
for idx, topic in lda_10.show_topics(formatted=False, num_words= 6):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))


Topic: 0 
Words: ['educacion', 'distancia', 'revista', 'digitales', 'ried', 'rt']
Topic: 1 
Words: ['educativa', 'rt', 'digital', 'innovacion', 'big', 'crear']
Topic: 2 
Words: ['salud', 'rt', 'ciencia', 'articulos', 'abierta', 'nunca']
Topic: 3 
Words: ['rt', 'herramientas', 'hacer', 'estudios', 'tic', 'uso']
Topic: 4 
Words: ['derecho', 'necesario', 'ejemplo', 'buscan', 'convierte', 'todas']
Topic: 5 
Words: ['rt', 'mejores', 'futuro', 'ideas', 'aprendizaje', 'inteligencia']
Topic: 6 
Words: ['rt', 'aprende', 'america', 'impacto', 'oportunidad', 'latina']
Topic: 7 
Words: ['rt', 'via', 'pais', 'digital', 'educacion', 'aprender']
Topic: 8 
Words: ['rt', 'revistas', 'venezuela', 'ucv', 'años', 'idiucv']
Topic: 9 
Words: ['rt', 'google', 'usar', 'educacion', 'atencion', 'tambien']
